In [ ]:
%%capture

import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from segment_speed_utils.project_vars import (RT_SCHED_GCS, 
                                              SCHED_GCS,
                                              GTFS_DATA_DICT
                                             )

import altair as alt

from IPython.display import HTML
from calitp_data_analysis import calitp_color_palette as cp

alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)


import great_tables as gt
from great_tables import md

import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)


In [ ]:
#name = "City of Santa Monica"

In [ ]:
%%capture_parameters
name

In [ ]:
FILE = GTFS_DATA_DICT.digest_tables.route_schedule_vp

df = pd.read_parquet(
    f"{RT_SCHED_GCS}{FILE}.parquet",
    filters = [[
        ("time_period", "==", "all_day"),
        ("organization_name", "==", name)]]
    
)

In [ ]:
display(df.typology.value_counts())

In [ ]:
# Import data 
most_recent_date = df.service_date.max()

# Operator data
operator_df = pd.read_parquet(
    f"{RT_SCHED_GCS}digest/operator_profiles.parquet",
    filters = [[
        ("organization_name", "==", name), 
        ("service_date", "==", most_recent_date)]]
)

# Operator route gdf to plot map
operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}digest/operator_routes.parquet",
    filters = [[
        ("organization_name", "==", name), 
        ("service_date", "==", most_recent_date)]]
)

In [ ]:
operator_route_gdf.dtypes

In [ ]:
def make_map(gdf: gpd.GeoDataFrame):
    cols = [c for c in gdf.columns if "is_" in c]
    m = gdf[["name", "route_combined_name", "geometry"] + cols].explore(
        "route_combined_name", 
        tiles = "CartoDB Positron", 
        legend=False
    )
    return m

In [ ]:
operator_p1 = [
    'operator_n_routes', 
    'operator_n_trips',
    'operator_n_shapes', 
    'operator_n_stops', 
]

operator_p2 = [
    'operator_n_arrivals',
    'operator_route_length_miles', 
    'operator_arrivals_per_stop'
]

operator_p3 = [
    'n_coverage_routes', 
    'n_downtown_local_routes', 
    'n_local_routes',
    'n_rapid_routes',
    'n_express_routes',
    'n_rail_routes'
]

def readable(column_name: str, readable_dict: dict) -> str:
    try:
        return readable_dict[column_name]["readable"]
    except:
        return readable_dict[column_name]
                             
def great_table_config(table: gt.GT) -> gt.GT:
    table = (table
             .tab_options(container_width = "75%")
             .tab_options(table_font_size="16px")
             .cols_align(align="center")
            )
    
    return table

In [ ]:
operator_p1_dict = {k: readable(k, readable_dict) for k in operator_p1}
operator_p2_dict = {k: readable(k, readable_dict) for k in operator_p2}
#operator_p3_dict = {k: readable(k, readable_dict) for k in operator_p3}

# {name}
## Operator Stats

In [ ]:
table1 = (gt.GT(data=operator_df[["name"] + operator_p1])
 .fmt_integer(
     columns = operator_p1,
     compact=True
 ).cols_align(align="center")
 .cols_label(**operator_p1_dict)
 .tab_header(
     title=md(f"### {operator_df.name.iloc[0]}: Daily Stats"),
     subtitle=md(f"#### {most_recent_date.date()}")
 )
)

table2 = (gt.GT(data=operator_df[operator_p2])
 .fmt_integer(
     columns = ["operator_n_arrivals"],
     compact=True
 ).cols_label(**operator_p2_dict)
.tab_source_note(
    source_note=md(
        "Service area (miles) is the the sum of miles across routes. "
        "<br>The longest shape is selected for each route."
    )
))
          
nacto_url = (
    "https://nacto.org/"
    "publication/transit-street-design-guide/"
    "introduction/service-context/transit-route-types/"
)

table3 = (gt.GT(data=operator_df[operator_p3])
 #.cols_label(**operator_p3_dict)
 ).tab_header(
     title=md(f"#### Route Typologies"),
     subtitle=md(f"#### Routes Classified in Each Typology")
 ).tab_source_note(
    source_note=md(
        f"Source: [NACTO Route Types]({nacto_url})"
        "<br>Transit routes can have multiple typologies. "
        "<br>A typology is selected by plurality."
    )
)

In [ ]:
display(great_table_config(table1))
display(great_table_config(table2))
display(great_table_config(table3))

In [ ]:
make_map(operator_route_gdf)

In [ ]:
operator_route_gdf.dtypes

In [ ]:
def base_route_chart(df: pd.DataFrame, y_col: str) -> alt.Chart:
    """
    """
    selected_colors = [
        cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
        cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
        cp.CALITP_CATEGORY_BOLD_COLORS[1], # orange,
    ]
    
    #https://stackoverflow.com/questions/26454649/python-round-up-to-the-nearest-ten

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
             x = alt.X("yearmonthdate(service_date):O", title = "Date",
                       axis = alt.Axis(format = '%b %Y')
                      ),
             y = alt.Y(f"{y_col}:Q"),
             color = alt.Color("time_period:N"),
             tooltip = ["route_combined_name", "route_id", "direction_id", 
                        "time_period", y_col]
         ).facet(
             column = alt.Column("direction_id:N"),
         ).interactive()
    )
    
    return chart

In [ ]:
# https://stackoverflow.com/questions/62103632/altair-change-the-position-of-a-slider
display(
    HTML(
        """
        <style>
        form.vega-bindings {
            position: absolute;
            right: 0px;
            top: 0px;
            }
        </style>
        """
    )
)

def filtered_route_charts(
    df: pd.DataFrame,
    control_field: str = "route_combined_name",
) -> alt.Chart:
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """

    route_dropdown = alt.binding_select(
        options=sorted(df[control_field].unique().tolist()), 
        name='Routes ', 
    )
        
    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=[control_field], 
        bind=route_dropdown,
    )
    
    vp_df = df[df.sched_rt_category != "schedule_only"]

    speeds_chart = base_route_chart(
        vp_df, "speed_mph"
    ).add_params(route_selector).transform_filter(route_selector)
    
    ping_density_chart = base_route_chart(
        vp_df, "vp_per_minute"
    ).add_params(route_selector).transform_filter(route_selector)
       
    
    chart_list = [
        speeds_chart,
        ping_density_chart, 
    ]
    
    chart = alt.vconcat(*chart_list).resolve_scale(y="independent")
    
    return chart


In [ ]:
available_typologies = df.typology.unique()
print(available_typologies)

In [ ]:
def make_chart(df, t: str):
    subset_df = df[df.typology==t]
    if len(subset_df) == 0:
        chart = alt.LayerChart()
    else:
        chart = filtered_route_charts(subset_df)
    
    return chart

## Downtown Local

In [ ]:
t = "downtown_local"
chart = make_chart(df, t)
chart

## Local

In [ ]:
t = "local"
chart = make_chart(df, t)
chart

## Coverage

In [ ]:
t = "coverage"
chart = make_chart(df, t)
chart

## Rapid

In [ ]:
t = "rapid"
chart = make_chart(df, t)
chart

## Express

In [ ]:
t = "express"
chart = make_chart(df, t)
chart

## Rail

In [ ]:
t = "rail"
chart = make_chart(df, t)
chart

## Unknown

In [ ]:
t = "unknown"
chart = make_chart(df, t)
chart